# Import Modules

In [1]:
import re
import threading
import time
import xlwings as xw

# Endnote Text File Formatting

In [2]:
class DataFormatting():
    """
    Export a tab-separated endnote-reference-list in a txt file format and open using excel.  Ensure the
    endnote output style headings match with the headings in the headings list (column_headings function). This 
    function will copy the references and apply formatting to improve asthetics and readabilty.  
    """
    def __init__(self, \
                 data_target_cell = "B3", \
                 title_font="Lato", \
                 content_font="Source Sans Pro", \
                 large_font_size = 16, \
                 small_font_size = 11):
        
        # Connect to excel 
        self.xlApp = xw.apps.active
        if self.xlApp is None: raise Exception("Excel must be open to continue!")
        
        # Test workbook connection
        self.connection_tested = False
        t1 = threading.Thread(target=self.connection_timer)
        t2 = threading.Thread(target=self.test_connection)
        t1.start(), t2.start()
        t1.join(), t2.join()
        
        # Connect to workbook
        if self.xlApp is None: raise Exception("Excels' data entry mode successfully deactivated.  Please rerun code.") 
        self.wb = xw.books.active  
    
        # Set range variables
        self.top_left_cell = data_target_cell
    
        # Font types and font sizes
        self.title_font =  title_font
        self.content_font = content_font
        self.large_font_size = large_font_size
        self.small_font_size = small_font_size
        
    
    def connection_timer(self):
        # Throws an error if test connection hangs
        time.sleep(0.1)
        if not self.connection_tested: 
            self.xlApp = None
            raise Exception("Excel is in data entry mode! Go to excel and press the 'esc' key to deactivate.")
            
            
    def test_connection(self):
        # Try connecting to the active workbook
        wb = xw.books.active
        wb = None
        self.connection_tested = True
    
    
    def get_data(self, ws_index):  
        # Select requested worksheet or maek a new worksheet
        try:
            self.ws = self.wb.sheets[ws_index]
        except Exception:
            raise Exception("The worksheet requiring formatting could not be found.")
        else:
            data_range = self.ws.used_range.address.replace("$", "").split(":")
            if len(data_range) < 1: raise Exception("No worksheet data to format.")
            return self.ws    
            
       
    def add_spacing_colunms(self):
        # Copy the initial data to a temp sheet 
        self.temp_sheet = self.wb.sheets.add(name=None, after=self.wb.sheets.count)
        self.temp_sheet.range("A1").value = self.ws.used_range.value
        
        # Add spacing columns 
        for col_indx in range(self.temp_sheet.used_range.shape[1], 0, -1):
            self.temp_sheet.range(1, col_indx).api.EntireColumn.Insert()
    
    
    def make_new_worksheet(self, ws_name):      
        # Copy the intial data (with spaces added) to a named sheet
        try:
            self.ws = self.wb.sheets.add(name=ws_name, after=self.wb.sheets.count)
        except:
            self.wb.sheets[ws_name].delete()
            self.ws = self.wb.sheets.add(name=ws_name, after=self.wb.sheets.count)
        finally:
            self.ws.range(self.top_left_cell).value = self.temp_sheet.used_range.value
            self.temp_sheet.delete()
    
    
    def data_coordinates(self):
        # Get data coordinates 
        data_range = self.ws.used_range.address.replace("$", "").split(":")
        
        # Top left cell and bottom right cell
        top_left_cell = self.top_left_cell
        bottom_right_cell = (data_range[1])
        print(f"The top left cell: {top_left_cell}")
        print(f"The bottom right cell: {bottom_right_cell}\n")
        
        # Top row and bottom row
        top_row = re.sub("[^0-9]", "", top_left_cell)        
        bottom_row = re.sub("[^0-9]", "", bottom_right_cell)
        print(f"Top worksheet row: {top_row}")
        print(f"Bottom worksheet row: {bottom_row}\n")
        
        # Left col and right col
        left_col = re.sub("[^a-zA-Z]", "", top_left_cell)     
        right_col = re.sub("[^a-zA-Z]", "", bottom_right_cell) 
        print(f"Left worksheet col: {left_col}")
        print(f"Right worksheet col: {right_col}\n")
        
        # Row and col count
        col_count = self.ws.range(f"{left_col}:{right_col}").columns.count
        row_count = (int(bottom_row) - int(top_row) +1)
        print(f"The col count is: {col_count}")
        print(f"The row count is: {row_count}\n")
        
        
        # Set range variables 
        self.headings_bar = self.ws.range(f"({left_col}2:{right_col}2)")
        self.headings_data = self.ws.range(f"{left_col}2:{right_col}2")
        self.table_data = self.ws.used_range
        print(f"The headings bar data: {self.headings_bar}")
        print(f"The headings data: {self.headings_data}")
        print(f"The table data: {self.table_data}")

        
    def column_headings(self, \
                        headings = ["Author", "Title", "Year", "Journal", "Abstract","Name of Database"], \
                        widths = [30, 61.43, 13.71, 51.71, 77.57, 30.14,]):
        # Insert blank-named spacing columns 
        for idx in range(len(headings), -1, -1):
            headings.insert(idx, "")
            widths.insert(idx, 0.5)
        
        # Add column headings / widths to worksheet 
        for idx in range(0, len(headings)):
            col_indx = idx +1
            self.ws.range(2, col_indx).value = headings[idx] 
            self.ws.range(2, col_indx).column_width = widths[idx]
            
    
    def format_worksheet(self):  
        # Font types
        self.headings_data.font.name = self.title_font
        self.table_data.font.name = self.content_font
        
        # Font sizes
        self.headings_data.font.size = self.large_font_size
        self.table_data.font.size = self.small_font_size
        
        # Font (bold, italic, wrap)
        self.headings_data.font.bold = True
        self.table_data.wrap_text = True
        
        # Font and background colour
        self.headings_bar.api.Interior.Color = 15921906 # nearly_white_1
       
        # Horizontal alignment
        self.headings_data.api.HorizontalAlignment = -4131  # xlHAlignLeft	
        self.table_data.api.HorizontalAlignment = -4131  # xlHAlignLeft
        
        # Vertical alignment
        self.headings_data.api.VerticalAlignment = -4108 # xlVAlignCenter	
        self.table_data.api.VerticalAlignment = -4160 # xlVAlignTop
        
        # Row heights
        self.ws.range("1:1").row_height = 5 
        self.headings_data.row_height = 24
        self.table_data.row_height = 18
    
        # Freeze titlebar and headings rows
        self.ws.range("3:3").select()
        self.xlApp.api.ActiveWindow.FreezePanes = True 
    
    
    def run(self):
        self.get_data(ws_index = 0)
        self.add_spacing_colunms()
        self.make_new_worksheet(ws_name="Cinhal")
        self.data_coordinates()
        self.column_headings()
        self.format_worksheet()
    
        
def main():  
    data_formatting = DataFormatting()
    data_formatting.run()    
    
if __name__=="__main__":
    main()   

The top left cell: B3
The bottom right cell: L49

Top worksheet row: 3
Bottom worksheet row: 49

Left worksheet col: B
Right worksheet col: L

The col count is: 11
The row count is: 47

The headings bar data: <Range [4a. Endnote Export Raw.xlsx]Cinhal!$B$2:$L$2>
The headings data: <Range [4a. Endnote Export Raw.xlsx]Cinhal!$B$2:$L$2>
The table data: <Range [4a. Endnote Export Raw.xlsx]Cinhal!$B$3:$L$49>
